In [32]:
import cv2
import numpy as np
import math

In [33]:
class Point(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def getX(self):
        return self.x

    def getY(self):
        return self.y


def selectConnects():
    """
    -1,-1:左上
    0,-1:正上
    1,-1:右上
    1,0:右侧
    1,1:右下
    0,1:正下
    -1,1:左下
    -1,0:左测
    """
    return [Point(-1, -1), Point(0, -1), Point(1, -1), Point(1, 0), Point(1, 1),
            Point(0, 1), Point(-1, 1), Point(-1, 0)]  # 八邻域


# 计算像素之间的偏差
def getGrayDiff(img, currentPoint, tmpPoint):
    return abs(int(img[currentPoint.x, currentPoint.y]) - int(img[tmpPoint.x, tmpPoint.y]))


def output_direction(point):
    if point.x == 0 and point.y == -1:
        return "↑"
    if point.x == 0 and point.y == 1:
        return "↓"
    if point.x == -1 and point.y == 0:
        return "←"
    if point.x == 1 and point.y == 0:
        return "→"
    if point.x == -1 and point.y == -1:
        return "↖"
    if point.x == 1 and point.y == -1:
        return "↗"
    if point.x == -1 and point.y == 1:
        return "↙"
    if point.x == 1 and point.y == 1:
        return "↘"


In [34]:
def region_grow(gray_image, seeds, thresh):
    # 读取图像的宽高，并建立一个和原图大小相同的seedMark
    height, weight = gray_image.shape
    seedMark = np.zeros(gray_image.shape, dtype=np.uint8)

    # 将定义的种子点放入种子点序列seedList
    seedList = []
    for seed in seeds:
        seedList.append(seed)
    label = 255
    label_count = 0

    # 选择邻域，更改程序默认8邻域
    connects = selectConnects()

    width = 0

    # 逐个点开始生长，生长的结束条件为种子序列为空，即没有生长点
    while len(seedList) > 0:
        # 弹出种子点序列的第一个点作为生长点
        currentPoint = seedList.pop(0)  # 弹出第一个元素
        # 并将生长点对应seedMark点赋值label（1），即为白色
        seedMark[currentPoint.x, currentPoint.y] = label
        label_count += 1

        # 以种子点为中心，八邻域的像素进行比较
        for i in range(len(connects)):
            # print(output_direction(connects[i]))
            tmpX = currentPoint.x + connects[i].x
            tmpY = currentPoint.y + connects[i].y
            # 判断是否为图像外的点，若是则跳过。  如果种子点是图像的边界点，邻域点就会落在图像外
            if tmpX < 0 or tmpY < 0 or tmpX >= height or tmpY >= weight:
                continue

            # 判断邻域点和种子点的差值
            grayDiff = getGrayDiff(gray_image, currentPoint, Point(tmpX, tmpY))
            if tmpY > currentPoint.y or tmpY < currentPoint.y:
                width += 1

            # 如果邻域点和种子点的差值小于阈值并且是没有被分类的点，则认为是和种子点同类，赋值label，
            # 并作为下一个种子点放入seedList
            if grayDiff < thresh and seedMark[tmpX, tmpY] == 0 and result[tmpX, tmpY] == 0:
                seedMark[tmpX, tmpY] = label
                # result[tmpX, tmpY] = label
                seedList.append(Point(tmpX, tmpY))
                label_count += 1

    print("label_count: ", label_count)
    return label_count, seedMark


In [36]:
image = cv2.imread("test.png", cv2.IMREAD_COLOR)
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

rows, cols = gray_image.shape
result = np.zeros(gray_image.shape)
region_imgs = []
region_dict = {}

for row in range(0, rows, 100):
    for col in range(0, cols, 100):
        if result[row, col] == 255 or gray_image[row, col] < 30:
            continue

        b, g, r = image[row, col]
        color_str = f'b[{b}], g[{g}], r[{r}]'

        seeds = [Point(row, col)]
        label_count, region_img = region_grow(gray_image, seeds, 5)
        if label_count < 271:
            continue
        region_imgs.append(region_img)

        if region_dict.get(color_str) is None:
            region_dict[color_str] = region_img
        else:
            region_dict[color_str] += region_img

for key in region_dict:
    img_path = "./segment/" + key + ".png"
    filtered_img = region_dict[key]
    # 将 img 生成数组
    # img_a = Image.fromarray(img)

    linesP = cv2.HoughLinesP(filtered_img, 1, np.pi / 180, threshold=5, minLineLength=1, maxLineGap=2)
    print(img_path)

    # 更新 linesP, 如果两点之间距离小于2，则过滤
    new_linesP = []
    for line in linesP:
        x1, y1, x2, y2 = line[0]
        if math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) > 2:
            new_linesP.append(line)
    #
    new_linesP = np.array(new_linesP)
    sorted_indices = np.lexsort((new_linesP[:, 0, 1], new_linesP[:, 0, 0]))
    new_linesP = new_linesP[sorted_indices]

    # 遍历
    # for line in new_linesP:
    #     print(line[0])

    cv2.imwrite(img_path, filtered_img)

label_count:  29783
label_count:  29783


KeyboardInterrupt: 